In [14]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib as plt
from itertools import chain
import ast

In [15]:
df = pd.read_csv("data/full_set.csv")
df.shape

(588793, 31)

In [16]:
TARGET_CLASS = "CHM2210"

In [17]:
# converts strings to arrays
df['Classes'] = df['Classes'].apply(ast.literal_eval)
df['Semester Grades'] = df['Semester Grades'].apply(ast.literal_eval)
df['Semester Points'] = df['Semester Points'].apply(ast.literal_eval)
df['CRN'] = df['CRN'].apply(ast.literal_eval)

In [18]:
# find all people who took class
Pidms_with_TARGET_CLASS = df[df['Classes'].apply(lambda x: TARGET_CLASS in x)]['Pidm'].unique()
df = df[df['Pidm'].isin(Pidms_with_TARGET_CLASS)]
df = df[['Pidm', 'Semester', 'HS GPA', 'Converted_SAT', 'Semester Points', 'Semester Grades', 'CRN', 'Classes']]
df.head(4)

Pidm  Semester  HS GPA  Converted_SAT         Semester Points  \
127  134328    201601     3.0         1370.0              [2.0, 3.0]   
128  134328    201701     3.0         1370.0                   [4.0]   
129  134328    201705     3.0         1370.0             [3.0, 3.67]   
130  134328    201708     3.0         1370.0  [1.0, 1.33, 0.0, 2.33]   

    Semester Grades                                   CRN  \
127          [C, B]                    [11143.0, 23005.0]   
128             [A]                             [11691.0]   
129         [B, A-]                    [52696.0, 52892.0]   
130  [D, D+, F, C+]  [86672.0, 87939.0, 88312.0, 91803.0]   

                                   Classes  
127                    [BSC2010L, BSC2010]  
128                              [ENC1101]  
129                     [PSY2012, ARC2211]  
130  [CHM2210, PCB3063, CHM2210L, EGS3720]

In [19]:
# Find the first semester with TARGET_CLASS for each student
def find_first_semester(student_df):
    chm2210_row = student_df[student_df['Classes'].apply(lambda x: TARGET_CLASS in x)]
    if not chm2210_row.empty:
        return chm2210_row['Semester'].min()
    return None

first_semester = df.groupby('Pidm').apply(lambda x: find_first_semester(x)).rename(f'{TARGET_CLASS}_Semester')
df = df.merge(first_semester, on='Pidm')

# filter all semesters after student took TARGET_CLASS
filtered_df = df[df['Semester'] <= df[f'{TARGET_CLASS}_Semester']]

In [20]:
# find grades/points for TARGET_CLASS and output it to a new column
def find_class_grades(student_df):
    for _, row in student_df.iterrows():
        if TARGET_CLASS in row['Classes']:
            index = row['Classes'].index(TARGET_CLASS)
            return row['Semester Points'][index], row['Semester Grades'][index]
    return None, None

class_grades = filtered_df.groupby('Pidm').apply(lambda x: find_class_grades(x)).apply(pd.Series)
class_grades.columns = [f'{TARGET_CLASS}_Points', f'{TARGET_CLASS}_Grade']

final_df = filtered_df.merge(class_grades, on='Pidm')

# filter out these grades
final_df = final_df[~final_df[f'{TARGET_CLASS}_Grade'].isin(['WE', 'IF', 'W', 'WC'])]


In [21]:
display(df[df['Pidm'] ==  134328])
final_df = final_df[final_df['Semester'] < final_df[f'{TARGET_CLASS}_Semester']]
display(final_df[final_df['Pidm'] ==  134328])
groupped_df = final_df.groupby('Pidm').agg({
    "HS GPA": 'first',
    'Converted_SAT': 'first',
    'Semester Grades': lambda x: sum(x, []),
    'Semester Points': lambda x: sum(x, []),
    'Classes': lambda x: sum(x, []),
    'CRN': lambda x: sum(x, []),
    f'{TARGET_CLASS}_Grade': 'first',
    f'{TARGET_CLASS}_Points': 'first',
}).reset_index()

Pidm  Semester  HS GPA  Converted_SAT         Semester Points  \
0  134328    201601     3.0         1370.0              [2.0, 3.0]   
1  134328    201701     3.0         1370.0                   [4.0]   
2  134328    201705     3.0         1370.0             [3.0, 3.67]   
3  134328    201708     3.0         1370.0  [1.0, 1.33, 0.0, 2.33]   
4  134328    201808     3.0         1370.0              [4.0, 0.0]   
5  134328    201901     3.0         1370.0         [1.0, 0.0, 0.0]   

  Semester Grades                                   CRN  \
0          [C, B]                    [11143.0, 23005.0]   
1             [A]                             [11691.0]   
2         [B, A-]                    [52696.0, 52892.0]   
3  [D, D+, F, C+]  [86672.0, 87939.0, 88312.0, 91803.0]   
4          [A, S]                    [80885.0, 89183.0]   
5       [D, F, F]           [17655.0, 18741.0, 19361.0]   

                                 Classes  CHM2210_Semester  
0                    [BSC2010L, BSC2010]            201708  
1                              [ENC1101]            201708  
2                     [PSY2012, ARC2211]            201708  
3  [CHM2210, PCB3063, CHM2210L, EGS3720]            201708  
4                     [COP2510, EGN3000]            201708  
5            [COP3514, COT3100, CDA3103]            201708

Pidm  Semester  HS GPA  Converted_SAT Semester Points Semester Grades  \
0  134328    201601     3.0         1370.0      [2.0, 3.0]          [C, B]   
1  134328    201701     3.0         1370.0           [4.0]             [A]   
2  134328    201705     3.0         1370.0     [3.0, 3.67]         [B, A-]   

                  CRN              Classes  CHM2210_Semester  CHM2210_Points  \
0  [11143.0, 23005.0]  [BSC2010L, BSC2010]            201708             1.0   
1           [11691.0]            [ENC1101]            201708             1.0   
2  [52696.0, 52892.0]   [PSY2012, ARC2211]            201708             1.0   

  CHM2210_Grade  
0             D  
1             D  
2             D

In [22]:
groupped_df.head(5)

Pidm  HS GPA  Converted_SAT  \
0   134328    3.00         1370.0   
1   706382    3.44         1280.0   
2  1085786    2.50         1180.0   
3  1096567    3.85         1340.0   
4  1302110    2.43         1040.0   

                                     Semester Grades  \
0                                   [C, B, A, B, A-]   
1                   [A, A+, A, WC, WC, WC, WC, A, A]   
2  [B, A, B, B+, A, W, W, B-, A, B, D, A, B-, A, ...   
3  [A-, A-, B-, W, A+, D, B, A+, B+, B, A, A+, A-...   
4                                                [A]   

                                     Semester Points  \
0                         [2.0, 3.0, 4.0, 3.0, 3.67]   
1      [4.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0]   
2  [3.0, 4.0, 3.0, 3.33, 4.0, 0.0, 0.0, 2.67, 4.0...   
3  [3.67, 3.67, 2.67, 0.0, 4.0, 1.0, 3.0, 4.0, 3....   
4                                              [4.0]   

                                             Classes  \
0     [BSC2010L, BSC2010, ENC1101, PSY2012, ARC2211]   
1  [AMH2020, CHM2045, CHM2045L, CHM2046, CHM2046L...   
2  [MAC2311, ECO2013, BSC2010, AMS4804, BSC2010L,...   
3  [MAC1147, ENC1102, CHM2045, CHM2045L, SLS2901,...   
4                                          [PCB3063]   

                                                 CRN CHM2210_Grade  \
0      [11143.0, 23005.0, 11691.0, 52696.0, 52892.0]             D   
1  [80953.0, 82346.0, 84033.0, 15234.0, 15241.0, ...             A   
2  [80534.0, 81300.0, 83081.0, 87275.0, 94320.0, ...            A-   
3  [82970.0, 89553.0, 92762.0, 92839.0, 93253.0, ...             C   
4                                          [54321.0]            A+   

   CHM2210_Points  
0            1.00  
1            4.00  
2            3.67  
3            2.00  
4            4.00

In [23]:
all_classes = sorted(set(chain.from_iterable(groupped_df['Classes'])))

def create_one_hot(classes, points, all_classes):
    one_hot_vector = [-1] * len(all_classes)
    for class_name, point in zip(classes, points):
        if class_name in all_classes:
            one_hot_vector[all_classes.index(class_name)] = point
    return one_hot_vector

groupped_df['One_Hot_Classes'] = groupped_df.apply(
    lambda row: create_one_hot(row['Classes'], row['Semester Points'], all_classes), axis=1
)

one_hot_df = pd.DataFrame(groupped_df['One_Hot_Classes'].tolist(), columns=all_classes, index=groupped_df['Pidm'])

In [24]:
print(set(groupped_df["CHM2210_Grade"]))

{'C-', 'S', 'C*', 'C-*', 'C', 'CF', 'B', 'D', 'C+', 'CC-', 'A+', 'B-*', 'D+', 'D*', 'B+', 'A-', 'U', 'F', 'CM', 'D-', 'F*', 'B-', 'A-*', 'FF', 'A'}


In [25]:
print(groupped_df.shape)
display(groupped_df.head(3))

print(one_hot_df.shape)
display(one_hot_df.head(3))

(11781, 10)


Pidm  HS GPA  Converted_SAT  \
0   134328    3.00         1370.0   
1   706382    3.44         1280.0   
2  1085786    2.50         1180.0   

                                     Semester Grades  \
0                                   [C, B, A, B, A-]   
1                   [A, A+, A, WC, WC, WC, WC, A, A]   
2  [B, A, B, B+, A, W, W, B-, A, B, D, A, B-, A, ...   

                                     Semester Points  \
0                         [2.0, 3.0, 4.0, 3.0, 3.67]   
1      [4.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0]   
2  [3.0, 4.0, 3.0, 3.33, 4.0, 0.0, 0.0, 2.67, 4.0...   

                                             Classes  \
0     [BSC2010L, BSC2010, ENC1101, PSY2012, ARC2211]   
1  [AMH2020, CHM2045, CHM2045L, CHM2046, CHM2046L...   
2  [MAC2311, ECO2013, BSC2010, AMS4804, BSC2010L,...   

                                                 CRN CHM2210_Grade  \
0      [11143.0, 23005.0, 11691.0, 52696.0, 52892.0]             D   
1  [80953.0, 82346.0, 84033.0, 15234.0, 15241.0, ...             A   
2  [80534.0, 81300.0, 83081.0, 87275.0, 94320.0, ...            A-   

   CHM2210_Points                                    One_Hot_Classes  
0            1.00  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...  
1            4.00  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...  
2            3.67  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...

(11781, 1524)


ACG2021  ACG2071  ACG3074  ACG3103  ACG3113  ACG3341  ACG3401  \
Pidm                                                                     
134328      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
706382      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
1085786     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         ACG4123  ACG4632  ACG4684  ACG5205  ADV2214  ADV3008  AFA2000  \
Pidm                                                                     
134328      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
706382      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
1085786     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         AFA2380  AFA3110  AFA4150  AFA4335  AFA4430  AFA4931  AFH3100  \
Pidm                                                                     
134328      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
706382      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
1085786     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         AFH3200  AFR1101  AFR1120  AFR2001  AFR2130  AFR2140  AFR3220  \
Pidm                                                                     
134328      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
706382      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
1085786     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         AFR3231  AFR4201  AFS2250  AMH2010  AMH2020  AMH3130  AMH3140  \
Pidm                                                                     
134328      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
706382      -1.0     -1.0     -1.0     -1.0      4.0     -1.0     -1.0   
1085786     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         AMH3160  AMH3170  AMH3270  AMH3390  AMH3402  AMH3403  AMH3421  \
Pidm                                                                     
134328      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
706382      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
1085786     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         AMH3423  AMH3533  AMH3545  AMH3571  AMH3572  AML3031  AML3051  \
Pidm                                                                     
134328      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
706382      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
1085786     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         AML3243  AML3604  AML3630  AML3674  AML4111  AMS2030  AMS2270  \
Pidm                                                                     
134328      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
706382      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
1085786     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         AMS3700  AMS3930  AMS4804  ANS3006  ANT2000  ANT2410  ANT2464  \
Pidm                                                                     
134328      -1.0     -1.0    -1.00     -1.0     -1.0     -1.0     -1.0   
706382      -1.0     -1.0    -1.00     -1.0     -1.0     -1.0     -1.0   
1085786     -1.0     -1.0     3.33     -1.0     -1.0     -1.0     -1.0   

         ANT2511  ANT2511L  ANT2721  ANT3005  ANT3101  ANT3610  ANT4012  \
Pidm                                                                      
134328      -1.0      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
706382      -1.0      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
1085786     -1.0      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         ANT4034  ANT4167  ANT4241  ANT4302  ANT4316  ANT4401  ANT4403  \
Pidm                                                                     
134328      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
706382      -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0  